In [1075]:
import numpy as np
import matplotlib.pyplot as plt

In [1076]:
def linear_interpolation(x0, y0, x1, y1, x):
    """
    Perform linear interpolation to find the y value corresponding to x.

    Parameters:
    x0, y0: coordinates of the first point
    x1, y1: coordinates of the second point
    x: the x value to interpolate

    Returns:
    y: the interpolated y value
    """
    y = y0 + (y1 - y0) * (x - x0) / (x1 - x0)
    return y

# Example usage:
# x0, y0 = 0, 0
# x1, y1 = 10, 10
# x = 5
# y = linear_interpolation(x0, y0, x1, y1, x)
# print(f"The interpolated value at x={x} is y={y}")

## Given Constants

In [1077]:
gear = "helix"


n_p = 1200

n_g_guess = 387

N_p = None # between 17 and 20 - only for spur?, idk abt helical
N_g = 0

P_d = None
P_nd = None

P = 5 # Power in HP
life_time = 15000 # in hours?


A_v = 10



In [1078]:
#helical
phi_n = None
phi_t = None

psi = 35

In [1079]:
#helical
if phi_n is None:
    phi_n = np.arctan(np.tan(phi_t * np.pi / 180) * np.cos(psi * np.pi / 180))
elif phi_t is None:
    phi_t = np.arctan(np.tan(phi_n * np.pi / 180) / np.cos(psi * np.pi / 180))

phi_n, phi_t

TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

Calculate P_d etc

In [1074]:
if P_d is None:
    P_d = P_nd * np.cos(psi * np.pi / 180)

if P_nd is None:
    P_nd = P_d / np.cos(psi * np.pi / 180)

P_x = np.pi / (P_d * np.tan(psi) )


## Calculations

In [1029]:
#shock factor
k_o = 1.5

In [1030]:
P_des = P * k_o
P_des

7.5

### Velocity ratio, output speed

In [1031]:
##Velocity ratio

VR_guess = n_p/n_g_guess


In [1032]:
##Number of teeth on gear

N_g = round(N_p * VR_guess)
N_g


59

In [1033]:
VR = N_g/N_p
n_g = n_p / VR

VR, n_g

(3.1052631578947367, 386.4406779661017)

### Diameters, Center distances, pitch line speed

In [1034]:
##Diameters

D_p = N_p / P_d
D_g = N_g / P_d

In [1035]:
##Center distance

C = (N_p + N_g) / (2 * P_d)
C

3.25

In [1036]:
##v_t

v_t = np.pi * D_p * n_p / 12
v_t

497.41883681838385

## Helical gear stuff follows

### Calculate J, I

In [ ]:
#find J_p and J_G

J_p = 0.34
J_G = 0.41

I = 0.1

assert(I > 0); assert(J_p > 0); assert(J_G > 0)

### Calculate forces

In [1037]:
## W_t - helical gear

W_t = 33000 * P / v_t
W_r = W_t * np.tan(phi_t)
Wx = W_t * np.tan(psi)
W_t, W_r, Wx

331.71240770731873

### Face width

In [1038]:
#face width

F = 2 * P_x #nominal value
F

1.0

## choose a material - this part is sketch

seems to apply the same for all gears??

this part consists of finding numbers in tables

In [1039]:
C_p = 2300 # for steel

In [1040]:
# usually given - use table 9-5 in mott 
# A_v = 10
assert(A_v is not None)


In [1041]:
k_v = 1.25 #from A_v

assert(k_v > 0)

In [1042]:
#C_pf
C_pf = None

if F / D_p < 0.5:
    assert(False)
    #use graph
elif F <= 1.0:
    C_pf = F / (10 * D_p) - 0.025
elif F <= 15:
    C_pf = F / (10 * D_p) - 0.0375 + 0.0125 * F

assert(C_pf is not None)

C_pf


0.038157894736842106

In [1043]:
#C_ma
cond = "commercial Enclosed".upper()
assert(cond is not None)

C_ma = -1

if cond == "commercial enclosed".upper():
    C_ma = 0.127 + 0.0158 * F - 1.093 * 10**-4 * F**2
elif cond == "precision".upper():
    C_ma = 0.0675 + 0.0128 * F - 0.926 * 10**-4 * F**2
elif cond == "extra".upper():
    C_ma = 0.0380 + 0.0102 * F - 0.822 * 10**-4 * F**2

C_ma


0.1426907

In [1044]:
k_m = 1 + C_ma + C_pf
k_m

1.1808485947368421

### Find $k_s, k_B, k_R, k_v, SF$ from a table idrk

In [1045]:
k_s = 1.0

In [1046]:
#k_B
# assert(False)
#idk how to determine this
k_B = 1.00
k_B

1.0

In [1047]:
K_R = 0

In [1048]:
safety_factor = 1.0

## Calculate $Y_{np}, Y_{ng}$ with the above numbers

### Number of loading cycles

In [1049]:
#Number of cycles

N_cp = 60 * life_time * n_p
N_cg = 60 * life_time * n_g

print(f"N_cp: {N_cp:.2e}, N_cg: {N_cg:.2e}")

N_cp: 1.08e+09, N_cg: 3.48e+08


### Calc $Y_{xx}, Z_{xx}$

In [1050]:
# assert(False)
Y_np = 0.95
Y_ng = 1.0

Z_np = 0.898
Z_ng = 0.922

## Calculate $S_{tP}$ and $S_{tG}$

In [1059]:
assert(F != 0)
assert(J_p != 0)

s_tp = (W_t * P_d) / (F * J_p) * k_o * k_s * k_m * k_B * k_v
s_tp

# W_t, P_d, F, J_p, k_o, k_s, k_m, k_B, k_v

25921.464518247463

In [1060]:
s_tG = s_tp * J_p / J_G
s_tG

21495.848624888145

In [1053]:
#idk what this is
s_atP_Greater_than = s_tp * k_R * safety_factor / Y_np
s_atP_Greater_than

27285.752124471015

In [1054]:
#idk what this is
s_atG_Greater_than = s_tG * k_R * safety_factor / Y_ng
s_atG_Greater_than
print("f")

f


### Calculate $S_c$

In [1055]:
S_c = C_p * np.sqrt( (W_t * k_o * k_s * k_m  * k_v) / (F * D_p * I) ) #
S_c

156646.3433519493

### Find adjusted values of $S_c$

In [1056]:
S_acP_Greater_than = S_c * (k_R * Safety_factor) / Z_np
S_acP_Greater_than

174439.13513580099

In [1057]:
S_acg_Greater_than = S_cp_Greater_than * (k_R * Safety_factor) / Z_ng
S_acg_Greater_than

117133.6534578091

In [1058]:
assert(False)

AssertionError: 